In [1]:
# https://www.mikulskibartosz.name/alternatives-to-open-ai-gpt-using-open-source-models-with-langchain/
!pip install transformers langchain google-search-results --quiet

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM


model_name = "cerebras/Cerebras-GPT-13B"
cache_dir = "/home/ec2-user/SageMaker/tmp"
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=cache_dir)
model = AutoModelForCausalLM.from_pretrained(model_name, cache_dir=cache_dir)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
from transformers import pipeline, Conversation

"""
'audio-classification', 'automatic-speech-recognition',
'conversational', 'depth-estimation', 'document-question-answering',
'feature-extraction', 'fill-mask', 'image-classification',
'image-segmentation', 'image-to-text', 'ner', 'object-detection',
'question-answering', 'sentiment-analysis', 'summarization',
'table-question-answering', 'text-classification', 'text-generation',
'text2text-generation', 'token-classification', 'translation',
'video-classification', 'visual-question-answering', 'vqa',
'zero-shot-audio-classification', 'zero-shot-classification',
'zero-shot-image-classification', 'zero-shot-object-detection', 'translation_XX_to_YY'
"""

pipe = pipeline(
    "text-generation", model=model, tokenizer=tokenizer,
    max_new_tokens=100, early_stopping=True, no_repeat_ngram_size=2
)

In [4]:
from langchain.llms import HuggingFacePipeline


llm = HuggingFacePipeline(pipeline=pipe)

In [5]:
# ・SimpleSequentialChain : 入出力を1つずつ持つチェーンを繋げるシンプルなチェーンです。
# ・SequentialChain : 入出力を複数持つチェーンを繋げる一般的なチェーン

from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

# 1つ目のチェーンの準備 : 劇のタイトルからあらすじを生成

# テンプレートの準備
template = """あなたは劇作家です。劇のタイトルが与えられた場合、そのタイトルのあらすじを書くのがあなたの仕事です。

タイトル:{title}
あらすじ:"""

# プロンプトテンプレートの準備
prompt_template = PromptTemplate(
    input_variables=["title"], 
    template=template
)

# LLMChainの準備
synopsis_chain = LLMChain(llm=llm, prompt=prompt_template)

In [7]:
# 2つ目のチェーン : 劇のあらすじからレビューを生成

# テンプレートの準備
template = """あなたは演劇評論家です。 劇のあらすじが与えられた場合、そのあらすじのレビューを書くのがあなたの仕事です。

あらすじ:
{synopsis}
レビュー:"""

# プロンプトテンプレートの準備
prompt_template = PromptTemplate(
    input_variables=["synopsis"], 
    template=template
)

# LLMChainの準備
review_chain = LLMChain(llm=llm, prompt=prompt_template)

In [8]:
from langchain.chains import SimpleSequentialChain

# SimpleSequentialChainで2つのチェーンを繋ぐ
overall_chain = SimpleSequentialChain(
    chains=[synopsis_chain, review_chain], 
    verbose=True
)

In [9]:
# SimpleSequentialChainの実行
review = overall_chain.run("浜辺の夕暮れ時の悲劇")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.




> Entering new SimpleSequentialChain chain...


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



海の中で、悔しい思い出を覚え込んでいる女の子。どうやって解放するのか。流れを変べるために、浮気していた男の姉と、彼女の苦しみを追いかけて、�
 
  ・第1章:私は、結婚したいと性格である。確かに素敵ではありますが、自分のことを知ったら、今はお互いに恋愛をしないでください。 

  この�

> Finished chain.


In [10]:
print(review)

 
  ・第1章:私は、結婚したいと性格である。確かに素敵ではありますが、自分のことを知ったら、今はお互いに恋愛をしないでください。 

  この�
